In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/Corpus_Christi/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/1301-easthaven-dr-corpus-christi-tx-78412-28828187',
 'https://www.trulia.com/home/218-hialeah-dr-corpus-christi-tx-78418-28855841',
 'https://www.trulia.com/home/14149-la-blanquilla-dr-corpus-christi-tx-78418-177375722',
 'https://www.trulia.com/home/2425-blessed-st-corpus-christi-tx-78418-345187649',
 'https://www.trulia.com/home/7213-nuss-dr-corpus-christi-tx-78414-122536837',
 'https://www.trulia.com/builder-community-plan/Ranch-View-Florence-2053901966',
 'https://www.trulia.com/home/5513-candace-st-corpus-christi-tx-78415-89007516',
 'https://www.trulia.com/home/2814-panay-dr-corpus-christi-tx-78418-28851033',
 'https://www.trulia.com/home/6737-sharks-ct-corpus-christi-tx-78414-249841965',
 'https://www.trulia.com/home/9309-moonglow-ln-corpus-christi-tx-78409-28849897',
 'https://www.trulia.com/home/10634-larkwood-st-corpus-christi-tx-78410-28860888',
 'https://www.trulia.com/home/14721-whitecap-blvd-164-corpus-christi-tx-78418-80250930',
 'https://w

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/1301-easthaven-dr-corpus-christi-tx-78412-28828187


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [15]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['1301 Easthaven Dr',
  '218 Hialeah Dr',
  '14149 La Blanquilla Dr',
  '2425 Blessed St',
  '7213 Nuss Dr',
  'Florence Plan in Ranch View by D.R. Horton - Corpus Christi',
  '5513 Candace St',
  '2814 Panay Dr',
  '6737 Sharks Ct',
  '9309 Moonglow Ln',
  '10634 Larkwood St',
  '14721 Whitecap Blvd #164',
  '6001 Pendleton Dr',
  '509 King St',
  '501 King St',
  '7406 Lake Windemere Dr',
  '4814 Jean St',
  '4026 Ogrady Dr',
  '813 Saint Benedict Ct',
  'Address Not Disclosed',
  '1330 Daytona Dr',
  '6418 Revolution Dr',
  '4802 Eider Dr',
  '15109 Dasmarinas Dr',
  '157 Sundew',
  '1250 Hayward Dr',
  '3609 Glenway St',
  '5026 Yorktown Blvd #4',
  '310 W  Vanderbilt Dr',
  '809 Saint Dominic Ct',
  '825 Indiana Ave',
  '2721 Cloyde St',
  '334 Melrose St',
  '6005 Tapestry Dr',
  '2209 Oak Crest St',
  '15209 S  Padre Island Dr #407',
  '3718 Fort Worth St',
  '14826 Aquarius St',
  '706 Snug Hbr',
  '908 11th St'],
 'Bathrooms': ['2 Baths',
  '1 Bath',
  '3 Baths',
 

In [16]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,1301 Easthaven Dr,2 Baths,"1,394 sqft",1968,Garage,"$265,000"
1,218 Hialeah Dr,1 Bath,"1,275 sqft",1963,No Info,"$184,900"
2,14149 La Blanquilla Dr,3 Baths,"2,712 sqft",2021,Garage,"$604,990"
3,2425 Blessed St,3 Baths,"2,536 sqft (on 0.30 acres)",2024,Garage,"$479,990"
4,7213 Nuss Dr,2 Baths,"1,460 sqft",2015,Garage,"$269,995"
5,Florence Plan in Ranch View by D.R. Horton - C...,2 Baths,"1,852 sqft",2024,Garage,"$294,900+"
6,5513 Candace St,2 Baths,"1,324 sqft",2007,Garage,"$224,900"
7,2814 Panay Dr,1 Bath,"1,083 sqft",1983,Garage,"$169,900"
8,6737 Sharks Ct,3 Baths,"2,294 sqft",2021,Garage,"$419,900"
9,9309 Moonglow Ln,2 Baths,"2,695 sqft",1980,Garage,"$295,000"


In [17]:
# Add location to dataframe
df['Location'] = 'Corpus Christi'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,1301 Easthaven Dr,2 Baths,"1,394 sqft",1968,Garage,"$265,000",Corpus Christi
1,218 Hialeah Dr,1 Bath,"1,275 sqft",1963,No Info,"$184,900",Corpus Christi
2,14149 La Blanquilla Dr,3 Baths,"2,712 sqft",2021,Garage,"$604,990",Corpus Christi
3,2425 Blessed St,3 Baths,"2,536 sqft (on 0.30 acres)",2024,Garage,"$479,990",Corpus Christi
4,7213 Nuss Dr,2 Baths,"1,460 sqft",2015,Garage,"$269,995",Corpus Christi
5,Florence Plan in Ranch View by D.R. Horton - C...,2 Baths,"1,852 sqft",2024,Garage,"$294,900+",Corpus Christi
6,5513 Candace St,2 Baths,"1,324 sqft",2007,Garage,"$224,900",Corpus Christi
7,2814 Panay Dr,1 Bath,"1,083 sqft",1983,Garage,"$169,900",Corpus Christi
8,6737 Sharks Ct,3 Baths,"2,294 sqft",2021,Garage,"$419,900",Corpus Christi
9,9309 Moonglow Ln,2 Baths,"2,695 sqft",1980,Garage,"$295,000",Corpus Christi


In [18]:
df.to_excel('dataframe_corpus_christi.xlsx', index=False)